# Unsupervised Anomaly Detection on Tennessee Eastman Process (TEP) Data using Transformer Autoencoder and Kalman Filter

**Objective:** This notebook implements an unsupervised anomaly detection system for the Tennessee Eastman Process (TEP) dataset. The system utilizes a Transformer-based autoencoder to learn normal operational behavior and a Kalman filter to smooth anomaly scores for more robust detection.

**Methodology:**
1.  **Data Loading and Preparation:**
    * Load fault-free training, fault-free testing, and faulty testing datasets for the TEP.
    * Identify unique fault types present in the faulty testing data.
2.  **Unsupervised Model Training:**
    * **Preprocessing:** Apply StandardScaler to the features.
    * **Time Series Windowing:** Convert the time series data into overlapping windows to serve as input sequences for the Transformer model.
    * **Model Architecture:**
        * **Transformer Autoencoder:** An autoencoder with Transformer encoder layers is used. It learns to reconstruct normal input sequences. Anomalies are expected to have higher reconstruction errors.
        * **Positional Encoding:** Added to the input embeddings to provide the model with information about the position of data points within a sequence.
    * **Training Loop:**
        * Train the autoencoder on fault-free training data to minimize reconstruction loss (Mean Squared Error).
        * Validate the model on a hold-out set of fault-free data.
        * Save model checkpoints periodically and the best performing model based on validation loss.
    * **Threshold Setting:** Determine an anomaly threshold based on the reconstruction errors from the fault-free validation data (e.g., 95th percentile).
    * **Kalman Filter:** A Kalman filter is initialized to smooth the raw reconstruction error scores, making anomaly detection less susceptible to noise.
3.  **Model Evaluation:**
    * Load the trained model and scaler.
    * If the threshold wasn't set or loaded properly, calculate it using fault-free test data.
    * For each specified fault type (including normal operation - fault type 0):
        * Prepare test data (scaling and windowing).
        * Generate predictions using the trained autoencoder and the determined threshold.
        * Apply the Kalman filter to the raw anomaly scores.
        * Calculate various performance metrics: Precision, Recall, F1-Score, Accuracy, ROC-AUC, PR-AUC, and components of the confusion matrix (TP, FP, TN, FN).
        * Generate and save visualizations:
            * Detailed evaluation plots (raw/filtered scores, true vs. predicted labels, score distributions).
            * Confusion matrix heatmaps.
            * ROC and Precision-Recall curves (if applicable).
    * Generate and save a summary report (CSV) and summary visualizations (F1 scores bar chart, metrics heatmap, confusion components bar charts) across all evaluated fault types.

**Libraries Used:**
* `matplotlib` for plotting.
* `numpy` for numerical operations.
* `pandas` for data manipulation.
* `pyreadr` for loading RData files.
* `torch` (PyTorch) for building and training the neural network.
* `gc` for garbage collection.
* `sklearn` for preprocessing (StandardScaler) and metrics.
* `tqdm` for progress bars.
* `os` for interacting with the file system.
* `time` for timing operations.
* `pickle` for saving/loading Python objects (like the scaler).
* `seaborn` for enhanced visualizations.
* `warnings` to manage warning messages.

## Model Training

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyreadr
import torch
import gc
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support, roc_auc_score, precision_recall_curve, auc
from tqdm import tqdm
import os
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

# Set device for PyTorch (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load datasets
df_train_faultfree = pyreadr.read_r("TEP_FaultFree_Training.RData")['fault_free_training']
df_test_faultfree = pyreadr.read_r("TEP_FaultFree_Testing.RData")['fault_free_testing']
df_test_faulty = pyreadr.read_r("TEP_Faulty_Testing.RData")['faulty_testing']

# Get all fault types
fault_types = sorted(df_test_faulty['faultNumber'].unique())
print(f"Available fault types: {fault_types}")

# Kalman Filter implementation for smoothing predictions
class KalmanFilter:
    """
    Simple implementation of Kalman filter for 1D signal smoothing
    """
    def __init__(self, dim_x, Q=1e-4, R=0.1):
        """
        Initialize Kalman filter

        Args:
            dim_x: State dimension
            Q: Process noise covariance
            R: Measurement noise covariance
        """
        self.dim_x = dim_x
        self.Q = Q
        self.R = R

    def filter(self, y_noisy):
        """
        Apply Kalman filtering to noisy observations

        Args:
            y_noisy: Noisy observations

        Returns:
            x_est: Filtered (smoothed) state estimates
        """
        n = len(y_noisy)
        x_est = np.zeros(n)  # State estimates
        P = np.zeros(n)      # Error covariance

        # Initial state and covariance
        if n > 0:
            x_est[0] = y_noisy[0]
            P[0] = 1.0
        else:
            return x_est # Return empty if no data

        for k in range(1, n):
            # Prediction step
            x_pred = x_est[k-1]      # State prediction
            P_pred = P[k-1] + self.Q  # Covariance prediction

            # Update step with measurement
            K = P_pred / (P_pred + self.R)  # Kalman gain
            x_est[k] = x_pred + K * (y_noisy[k] - x_pred)  # State update
            P[k] = (1 - K) * P_pred    # Covariance update

        return x_est

# Create PyTorch dataset
class UnsupervisedDataset(Dataset):
    """
    Dataset for unsupervised learning (no labels)
    """
    def __init__(self, X):
        self.X = X

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]


# Create sliding window time series input
def create_time_series_windows(X, lookback=15, max_samples=None, start_idx=0):
    """
    Create sliding window dataset for unsupervised learning
    with memory efficiency optimizations and proper handling of time order
    
    This modified version ensures no data leakage across time boundaries
    
    Args:
        X: Features array
        lookback: Number of time steps to look back
        max_samples: Maximum number of samples to use (for memory constrained environments)
        start_idx: Start index to avoid overlapping with other datasets (prevents data leakage)
        
    Returns:
        X_windows: Tensor of shape (n_samples, lookback, n_features)
    """
    
    # Calculate valid range for window creation (respecting time boundaries)
    # Number of windows that can be created.
    n_potential_samples = len(X) - lookback + 1 - start_idx
    # Ensure n_potential_samples is not negative
    n_samples = max(0, n_potential_samples)


    # Handle max_samples limit if specified
    if max_samples is not None and n_samples > max_samples:
        # Take consecutive samples in time order (NOT random sampling)
        n_samples = max_samples
    
    # Pre-allocate memory for efficiency
    n_features = X.shape[1]
    X_windows = np.zeros((n_samples, lookback, n_features), dtype=np.float32)
    
    # Process in batches to save memory ( tqdm original had range(0, n_samples, batch_size) )
    # Corrected loop for creating windows: iterate up to n_samples
    for i in tqdm(range(n_samples), desc="Creating windows", miniters=int(n_samples/100) if n_samples > 100 else 1, leave=False):
        actual_idx = start_idx + i
        X_windows[i] = X[actual_idx : actual_idx + lookback]
    
    return torch.tensor(X_windows, dtype=torch.float32)


# Positional Encoding for Transformer model
class PositionalEncoding(nn.Module):
    """
    Positional encoding for Transformer model
    Adds information about position of tokens in the sequence
    """
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Create positional encoding
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        """
        Add positional encoding to input
        
        Args:
            x: Input tensor of shape (batch_size, seq_length, d_model)
            
        Returns:
            Tensor with positional encoding added
        """
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


# Transformer-based autoencoder for unsupervised anomaly detection
class TransformerAutoencoder(nn.Module):
    """
    Transformer-based autoencoder for unsupervised anomaly detection
    """
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2, dropout=0.1):
        """
        Initialize Transformer autoencoder
        
        Args:
            input_dim: Input feature dimension
            d_model: Hidden dimension for the model
            nhead: Number of attention heads
            num_layers: Number of transformer encoder layers
            dropout: Dropout probability
        """
        super().__init__()
        
        # Input embedding
        self.embedding = nn.Linear(input_dim, d_model)
        
        # Positional encoding
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        # Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=256, 
            dropout=dropout, 
            batch_first=True
        )
        
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Decoder to reconstruct the input
        self.decoder = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )
        
    def forward(self, x):
        """
        Forward pass of the model
        
        Args:
            x: Input tensor of shape (batch_size, seq_length, input_dim)
            
        Returns:
            x_reconstructed: Reconstructed input
        """
        batch_size, seq_len, _ = x.size()
        
        # Embedding and scale
        x_embedded = self.embedding(x) * np.sqrt(self.embedding.out_features)
        
        # Add positional encoding
        x_embedded = self.pos_encoder(x_embedded)
        
        # Pass through transformer encoder
        x_encoded = self.transformer_encoder(x_embedded)
        
        # Decode each time step
        x_reconstructed = torch.zeros_like(x)
        for i in range(seq_len):
            x_reconstructed[:, i] = self.decoder(x_encoded[:, i])
            
        return x_reconstructed
    
    def get_reconstruction_error(self, x):
        """
        Calculate reconstruction error (MSE) for anomaly detection
        with memory-efficient implementation
        
        Args:
            x: Input tensor
            
        Returns:
            error: Reconstruction error
        """
        # Process one sample at a time if batch size is 1
        if x.size(0) == 1:
            x_reconstructed = self(x)
            error = torch.mean(torch.pow(x - x_reconstructed, 2), dim=(1, 2))
            return error
        
        # For larger batches, split into smaller sub-batches if needed
        batch_size = x.size(0)
        if batch_size > 4:  # If batch is large
            errors = []
            sub_batch_size = 4  # Process 4 samples at a time
            
            for i in range(0, batch_size, sub_batch_size):
                end_idx = min(i + sub_batch_size, batch_size)
                sub_batch = x[i:end_idx]
                sub_reconstructed = self(sub_batch)
                sub_error = torch.mean(torch.pow(sub_batch - sub_reconstructed, 2), dim=(1, 2))
                errors.append(sub_error)
                
                # Free memory
                del sub_batch, sub_reconstructed
                
            return torch.cat(errors)
        else:
            # Standard processing for reasonable batch sizes
            x_reconstructed = self(x)
            error = torch.mean(torch.pow(x - x_reconstructed, 2), dim=(1, 2))
            return error


# Neural network anomaly detector with reconstruction-based approach
class UnsupervisedAnomalyDetector:
    """
    Unsupervised anomaly detector based on reconstruction error
    """
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2, dropout=0.1, Q=1e-4, R=0.1):
        """
        Initialize detector
        
        Args:
            input_dim: Input feature dimension
            d_model: Hidden dimension for transformer
            nhead: Number of attention heads
            num_layers: Number of transformer layers
            dropout: Dropout probability
            Q: Process noise for Kalman filter
            R: Measurement noise for Kalman filter
        """
        self.autoencoder = TransformerAutoencoder(input_dim, d_model, nhead, num_layers, dropout)
        self.kalman_filter = KalmanFilter(1, Q, R)
        self.device = device
        self.autoencoder.to(self.device)
        
        # Training related variables
        self.train_losses = []
        self.val_losses = []
        self.epochs = []
        self.current_epoch = 0 # Stores the last completed epoch (0-indexed)
        
        # Threshold for anomaly detection
        self.threshold = None
        
        # Checkpoint directory
        self.checkpoint_dir = 'checkpoints'
        os.makedirs(self.checkpoint_dir, exist_ok=True)
        
    def save_checkpoint(self, epoch, optimizer, loss, filename=None):
        """
        Save model checkpoint - safely handles None optimizer
        
        Args:
            epoch: Current epoch (1-based for saving convention)
            optimizer: Optimizer state (can be None for inference-only checkpoints)
            loss: Current loss value
            filename: Name of the checkpoint file
        """
        if filename is None:
            filename = f'unsupervised_checkpoint.pth'
        path = os.path.join(self.checkpoint_dir, filename)
    
        # Create checkpoint dictionary
        checkpoint_dict = {
            'epoch': epoch, # This is the 1-based epoch number
            'model_state_dict': self.autoencoder.state_dict(),
            'loss': loss,
            'train_losses': self.train_losses,
            'val_losses': self.val_losses,
            'epochs': self.epochs, # List of 1-based epoch numbers trained
            'current_epoch': self.current_epoch, # Last completed 0-indexed epoch
            'threshold': self.threshold
        }
    
        # Only add optimizer state if it's not None
        if optimizer is not None:
            try:
                checkpoint_dict['optimizer_state_dict'] = optimizer.state_dict()
            except AttributeError:
                print("Warning: optimizer does not have state_dict method, not saving optimizer state")
    
        torch.save(checkpoint_dict, path)
        print(f"Checkpoint saved at {path}")
        
    def load_checkpoint(self, optimizer, filename=None):
        """
        Load model checkpoint
        
        Args:
            optimizer: Optimizer to load state into (can be None)
            filename: Name of the checkpoint file
            
        Returns:
            start_epoch: Epoch to start training from (0-indexed for loop)
            loss: Loss value of the checkpoint
        """
        if filename is None:
            filename = f'unsupervised_checkpoint.pth'
        path = os.path.join(self.checkpoint_dir, filename)
        if os.path.exists(path):
            checkpoint = torch.load(path, map_location=self.device)
            self.autoencoder.load_state_dict(checkpoint['model_state_dict'])
            
            if optimizer is not None and 'optimizer_state_dict' in checkpoint:
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                
            self.train_losses = checkpoint.get('train_losses', [])
            self.val_losses = checkpoint.get('val_losses', [])
            self.epochs = checkpoint.get('epochs', [])
            self.current_epoch = checkpoint.get('current_epoch', 0) # Last completed 0-indexed epoch
            self.threshold = checkpoint.get('threshold', None)
            
            # 'epoch' in checkpoint is 1-based, so start_epoch for training loop is this value
            # current_epoch (0-indexed) is more reliable for determining next epoch.
            start_epoch_for_loop = self.current_epoch + 1 if self.epochs else 0


            print(f"Checkpoint loaded from {path}, resuming from epoch {start_epoch_for_loop}")
            return start_epoch_for_loop, checkpoint.get('loss', float('inf'))
        else:
            print(f"No checkpoint found at {path}, starting from scratch")
            return 0, float('inf') # Start from epoch 0 (0-indexed)
        
    def validate(self, val_loader):
        """
        Validate model on validation set
        
        Args:
            val_loader: Validation data loader
            
        Returns:
            avg_val_loss: Average validation loss
        """
        self.autoencoder.eval()
        total_loss = 0.0
        
        with torch.no_grad():
            for X_batch in val_loader:
                X_batch = X_batch.to(self.device)
                X_reconstructed = self.autoencoder(X_batch)
                
                # MSE reconstruction loss
                loss = torch.mean(torch.pow(X_batch - X_reconstructed, 2))
                total_loss += loss.item()
                
        avg_val_loss = total_loss / len(val_loader) if len(val_loader) > 0 else 0.0
        return avg_val_loss
        
    def train_model(self, train_loader, val_loader=None, num_epochs=30, lr=1e-3, resume=True, save_interval=5):
        """
        Train the model with checkpoint support and real-time plotting
        
        Args:
            train_loader: Training data loader (normal samples only)
            val_loader: Validation data loader
            num_epochs: Number of epochs to train in this session
            lr: Learning rate
            resume: Whether to resume from checkpoint
            save_interval: Interval to save checkpoints
        """
        optimizer = torch.optim.Adam(self.autoencoder.parameters(), lr=lr)
        
        start_epoch_idx = 0 # This will be the 0-indexed epoch to start the loop from
        best_loss = float('inf')
        if resume:
            start_epoch_idx, best_loss = self.load_checkpoint(optimizer)
            # self.current_epoch is already updated by load_checkpoint to the last *completed* 0-indexed epoch
        
        if start_epoch_idx == 0 and not self.epochs: # if not resuming or no history
            self.train_losses = []
            self.val_losses = []
            self.epochs = []
            self.current_epoch = -1 # Indicates no epochs completed yet
        
        self.autoencoder.train()
        training_interrupted = False
        # The loop runs for `num_epochs` iterations.
        # `epoch_iter` goes from 0 to `num_epochs - 1`.
        # `current_training_epoch` is the actual epoch number (0-indexed globally).
        for epoch_iter in range(num_epochs):
            current_training_epoch = start_epoch_idx + epoch_iter # Current 0-indexed epoch number
            self.current_epoch = current_training_epoch
            epoch_start_time = time.time()
            total_loss = 0.0
            
            batch_pbar = tqdm(train_loader, desc=f"Epoch {current_training_epoch+1}/{start_epoch_idx + num_epochs}", leave=False)
            
            try:
                for X_batch in batch_pbar:
                    X_batch = X_batch.to(self.device)
                    optimizer.zero_grad()
                    X_reconstructed = self.autoencoder(X_batch)
                    loss = torch.mean(torch.pow(X_batch - X_reconstructed, 2))
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                    batch_pbar.set_postfix({"batch_loss": f"{loss.item():.4f}"})
            
                avg_train_loss = total_loss / len(train_loader) if len(train_loader) > 0 else 0.0
                self.train_losses.append(avg_train_loss)
                self.epochs.append(current_training_epoch + 1) # Store 1-based epoch number
                
                avg_val_loss = None
                if val_loader:
                    avg_val_loss = self.validate(val_loader)
                    self.val_losses.append(avg_val_loss)
                
                epoch_time = time.time() - epoch_start_time
                status = f"Epoch {current_training_epoch+1}/{start_epoch_idx + num_epochs}, Train Loss: {avg_train_loss:.4f}"
                if avg_val_loss is not None:
                    status += f", Val Loss: {avg_val_loss:.4f}"
                status += f", Time: {epoch_time:.2f}s"
                print(status)
                
                if (current_training_epoch + 1) % 5 == 0: # Plot every 5 epochs
                    plt.figure(figsize=(10, 6))
                    # Ensure self.epochs is used as x-axis for both train and val losses
                    # and that val_losses are aligned with epochs if resuming
                    
                    # Align train_losses with self.epochs
                    plot_epochs = self.epochs
                    plot_train_losses = self.train_losses[-len(plot_epochs):] # ensure alignment if lists got desynced
                                                                            # (should not happen with current logic)

                    plt.plot(plot_epochs, plot_train_losses, 'b-', label='Training Loss')

                    if self.val_losses:
                        # Determine the correct x-values for validation losses based on when they were recorded
                        # Assuming one val_loss per epoch stored in self.epochs
                        if len(self.val_losses) <= len(plot_epochs):
                             # If val_losses is shorter or equal, align from the end
                            val_epochs_x = plot_epochs[-len(self.val_losses):]
                            plot_val_losses = self.val_losses
                        else: # Should not happen if val_loss appended each epoch
                            val_epochs_x = plot_epochs 
                            plot_val_losses = self.val_losses[-len(val_epochs_x):]

                        if len(val_epochs_x) == len(plot_val_losses) and len(val_epochs_x) > 0 :
                             plt.plot(val_epochs_x, plot_val_losses, 'r-', label='Validation Loss')


                    plt.xlabel('Epoch')
                    plt.ylabel('Loss')
                    plt.title('Reconstruction Loss')
                    plt.legend()
                    plt.grid(True)
                    plt.savefig(f'unsupervised_loss_epoch_{current_training_epoch+1}.png')
                    plt.close()
                
                if (current_training_epoch + 1) % save_interval == 0:
                    self.save_checkpoint(current_training_epoch + 1, optimizer, avg_train_loss, f'unsupervised_checkpoint_epoch_{current_training_epoch+1}.pth')
                
                current_eval_loss = avg_val_loss if avg_val_loss is not None else avg_train_loss
                if current_eval_loss < best_loss:
                    best_loss = current_eval_loss
                    self.save_checkpoint(current_training_epoch + 1, optimizer, best_loss, f'unsupervised_best_model.pth')
                    print(f"New best model saved with loss: {best_loss:.4f} at epoch {current_training_epoch+1}")

            except KeyboardInterrupt:
                print(f"\nTraining interrupted by user at epoch {current_training_epoch+1}! Saving checkpoint...")
                training_interrupted = True
                # Calculate loss for completed batches in the interrupted epoch
                num_batches_done = len(batch_pbar) - batch_pbar.n # remaining batches
                num_batches_done = len(train_loader) - num_batches_done if num_batches_done <= len(train_loader) else len(train_loader)

                last_loss = total_loss / num_batches_done if num_batches_done > 0 else float('inf')
                self.save_checkpoint(current_training_epoch + 1, optimizer, last_loss, f'unsupervised_interrupted_checkpoint.pth')
                break # Exit the epoch loop
            
            if training_interrupted:
                break # Exit the outer loop if interrupted

        if not training_interrupted and self.epochs: # Ensure training ran for at least one epoch
            final_epoch_num = self.epochs[-1] 
            final_loss = self.train_losses[-1]
            self.save_checkpoint(final_epoch_num, optimizer, final_loss, f'unsupervised_final_model.pth')
            print(f"Training complete! Final model saved after epoch {final_epoch_num}.")
        elif not self.epochs:
             print("Training did not run for any epochs (e.g., num_epochs was 0 or loaded checkpoint was at target). No final model saved from this run.")
        else: # training_interrupted
            print("Training was interrupted. Final model not saved as 'unsupervised_final_model.pth'. Interrupted checkpoint saved.")
        
        return optimizer
    
    def set_threshold(self, validation_loader, percentile=95):
        """
        Set threshold for anomaly detection based on validation data
        
        Args:
            validation_loader: Validation data loader (normal samples)
            percentile: Percentile to set as threshold
        """
        self.autoencoder.eval()
        errors = []
        
        if len(validation_loader) == 0:
            print("Warning: Validation loader is empty. Cannot set threshold. Using a default or previously set threshold.")
            if self.threshold is None: # if no threshold was ever set (e.g. from checkpoint)
                self.threshold = 1.0 # Arbitrary default, should be tuned or error out
                print("Setting threshold to arbitrary 1.0 due to empty validation set.")
            return

        with torch.no_grad():
            for X_batch in tqdm(validation_loader, desc="Computing threshold"):
                X_batch = X_batch.to(self.device)
                reconstruction_errors = self.autoencoder.get_reconstruction_error(X_batch)
                errors.extend(reconstruction_errors.cpu().numpy())
        
        if not errors:
            print("Warning: No reconstruction errors computed from validation set. Threshold not set. Using previous or default.")
            if self.threshold is None:
                self.threshold = 1.0
                print("Setting threshold to arbitrary 1.0.")
            return

        self.threshold = np.percentile(errors, percentile)
        print(f"Threshold set to {self.threshold:.4f} based on {percentile}th percentile of validation errors.")
        
    def predict(self, X, batch_size=4):
        """
        Make predictions using the reconstruction error and apply Kalman filtering
        with memory-efficient batch processing
        
        Args:
            X: Input data tensor or numpy array
            batch_size: Batch size for prediction to avoid OOM errors
            
        Returns:
            anomalies: Binary anomaly predictions
            filtered_scores: Continuous anomaly scores after Kalman filtering
            raw_scores: Raw reconstruction errors
        """
        # Adhering to the original script's structure for this part:
        if self.threshold is None:
            # The original script had an indented self.autoencoder.eval() here.
            # This implies it's part of the if block.
            self.autoencoder.eval()
            # It's critical to note that predicting with a None threshold will cause issues later.
            # A proper check should raise an error.
            print("Warning: UnsupervisedAnomalyDetector.predict called with self.threshold = None. This will likely lead to errors or incorrect behavior.")
        
        self.autoencoder.eval() # This was the unconditional eval call in the original script.

        # A more robust check, though not in original, would be:
        if self.threshold is None:
             raise ValueError("Threshold is not set. Call set_threshold or load a checkpoint with a threshold before predicting.")


        if isinstance(X, np.ndarray):
            total_samples = X.shape[0]
        elif isinstance(X, torch.Tensor):
            total_samples = X.size(0)
        else:
            raise TypeError("Input X must be a NumPy array or PyTorch Tensor.")

        if total_samples == 0:
            return np.array([]), np.array([]), np.array([])

        all_errors = np.zeros(total_samples)
        
        try:
            with torch.no_grad():
                for i in tqdm(range(0, total_samples, batch_size), desc="Predicting", leave=False):
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    
                    end_idx = min(i + batch_size, total_samples)
                    if isinstance(X, np.ndarray):
                        X_batch_np = X[i:end_idx]
                        X_tensor = torch.tensor(X_batch_np, dtype=torch.float32).to(self.device)
                    else: # X is already a Tensor
                        X_tensor = X[i:end_idx].to(self.device)
                    
                    reconstruction_errors = self.autoencoder.get_reconstruction_error(X_tensor)
                    all_errors[i:end_idx] = reconstruction_errors.cpu().numpy()
                    
                    del X_tensor, reconstruction_errors
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
            
            filtered_scores = self.kalman_filter.filter(all_errors)
            anomalies = filtered_scores > self.threshold # This will error if self.threshold is None
            
            return anomalies, filtered_scores, all_errors
            
        except Exception as e:
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            raise e

def train_unsupervised(df_train_faultfree, lookback=15, num_epochs=30, batch_size=16):
    """
    Train unsupervised model and save the model and scaler
    """
    try:
        print(f"\n{'='*80}")
        print(f"Training unsupervised anomaly detector")
        print(f"{'='*80}")
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        X_train_raw = df_train_faultfree.iloc[:, 3:].values
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_raw)
        
        train_val_split_idx = int(0.9 * len(X_train_scaled))
        X_train_for_model = X_train_scaled[:train_val_split_idx]
        X_val_for_model = X_train_scaled[train_val_split_idx:]
        
        X_train_windows = create_time_series_windows(X_train_for_model, lookback, start_idx=0)
        X_val_windows = create_time_series_windows(X_val_for_model, lookback, start_idx=0)
        
        # Handle case where window creation might yield empty tensors (e.g., if data is too short)
        if X_train_windows.nelement() == 0 and X_val_windows.nelement() == 0:
            print("Error: Not enough training/validation data to create any windows. Aborting training.")
            return None, None
        if X_train_windows.nelement() == 0:
            print("Warning: No training windows created. Check training data length and lookback.")
            # Potentially use validation windows for feature_dim if training is empty, though this is unusual.
            if X_val_windows.nelement() == 0:
                 print("Error: No validation windows either. Cannot determine feature dimension.")
                 return None, None

        train_dataset = UnsupervisedDataset(X_train_windows)
        val_dataset = UnsupervisedDataset(X_val_windows)
        
        # Use larger batch_size for validation loader if possible, but original used training batch_size
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True if len(train_dataset) > batch_size else False)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False) 
        
        # Determine feature_dim robustly
        if X_train_windows.nelement() > 0:
            feature_dim = X_train_windows.shape[2]
        elif X_val_windows.nelement() > 0: # Fallback to val windows if train is empty
            feature_dim = X_val_windows.shape[2]
            print("Warning: Using validation window features to determine model input dimension as training windows are empty.")
        else: # Fallback to raw data shape if no windows at all (should have been caught earlier)
            feature_dim = df_train_faultfree.shape[1] - 3 
            print("Critical Warning: No windows created. Using raw feature count for model input dimension. This may be incorrect.")

        detector = UnsupervisedAnomalyDetector(
            input_dim=feature_dim, d_model=64, nhead=4, num_layers=2, dropout=0.1, Q=1e-5, R=0.1
        )
        
        del X_train_raw, X_train_scaled, X_train_for_model, X_val_for_model, X_train_windows, X_val_windows, train_dataset, val_dataset
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        optimizer = detector.train_model( 
            train_loader=train_loader, val_loader=val_loader if len(val_loader)>0 else None, num_epochs=num_epochs,
            lr=1e-3, resume=True, save_interval=5
        )
        
        if len(val_loader)>0:
            detector.set_threshold(val_loader, percentile=95) 
        else: # No validation data to set threshold
            print("Warning: Validation loader is empty. Cannot set threshold automatically.")
            # Attempt to load threshold from 'best_model' or 'final_model' if they exist and have one
            # Or use a pre-defined default, or raise an error.
            # For now, if it's None, predict() will raise error.
            loaded_epoch, loaded_loss = detector.load_checkpoint(optimizer, filename='unsupervised_best_model.pth') # try loading best
            if detector.threshold is None:
                loaded_epoch, loaded_loss = detector.load_checkpoint(optimizer, filename='unsupervised_final_model.pth') # try final
            if detector.threshold is None:
                print("Could not set threshold from validation data or load from checkpoints. Manual setting or default needed.")
                # detector.threshold = some_default_value # e.g. np.percentile of reconstruction errors on a small normal test subset
                                                       # Or raise error before evaluation loop.
                # For now, allow it to be None and let predict() handle it.

        try:
            epoch_for_save = detector.epochs[-1] if detector.epochs else (detector.current_epoch + 1 if detector.current_epoch is not None and detector.current_epoch !=-1 else 0)
            loss_for_save = (detector.val_losses[-1] if detector.val_losses else 
                            (detector.train_losses[-1] if detector.train_losses else 0.0))
            detector.save_checkpoint(epoch_for_save, optimizer, loss_for_save, 
                                    f'unsupervised_final_model_with_threshold.pth')
        except Exception as e:
            print(f"Warning: Could not save final model with threshold: {str(e)}")
        
        # Save the scaler for later use
        with open('scaler.pkl', 'wb') as f:
            pickle.dump(scaler, f)
            print("StandardScaler saved to scaler.pkl")
            
        print("\n" + "="*80)
        print("Unsupervised model training run complete.")
        print("Model has been saved in the checkpoints directory.")
        print("="*80)
    
        return detector, scaler

    except Exception as e:
        print(f"Critical error in training function: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None

# Main training function
def main_training():
    """
    Main function to train unsupervised model
    """
    lookback = 15
    num_epochs = 30  # Can be adjusted
    batch_size = 16  # For training

    try:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
        detector, scaler = train_unsupervised(
            df_train_faultfree,
            lookback=lookback,
            num_epochs=num_epochs,
            batch_size=batch_size
        )
        
        return detector, scaler

    except Exception as e:
        print(f"Critical error in main training function: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None

# Only execute main function if script is run directly
if __name__ == "__main__":
    detector, scaler = main_training()

Using device: cuda
Available fault types: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

Training unsupervised anomaly detector


No checkpoint found at checkpoints\unsupervised_checkpoint.pth, starting from scratch


Epoch 1/30, Train Loss: 0.0567, Val Loss: 0.0046, Time: 320.71s
Checkpoint saved at checkpoints\unsupervised_best_model.pth
New best model saved with loss: 0.0046 at epoch 1


Epoch 2/30, Train Loss: 0.0006, Val Loss: 0.0003, Time: 317.41s
Checkpoint saved at checkpoints\unsupervised_best_model.pth
New best model saved with loss: 0.0003 at epoch 2


Epoch 3/30, Train Loss: 0.0002, Val Loss: 0.0001, Time: 317.62s
Checkpoint saved at checkpoints\unsupervised_best_model.pth
New best model saved with loss: 0.0001 at epoch 3


...

Epoch 28/30, Train Loss: 0.0000, Val Loss: 0.0000, Time: 326.65s


Epoch 29/30, Train Loss: 0.0001, Val Loss: 0.0000, Time: 326.38s
Checkpoint saved at checkpoints\unsupervised_best_model.pth
New best model saved with loss: 0.0000 at epoch 29


Epoch 30/30, Train Loss: 0.0000, Val Loss: 0.0000, Time: 326.64s
Checkpoint saved at checkpoints\unsupervised_checkpoint_epoch_30.pth
Checkpoint saved at checkpoints\unsupervised_final_model.pth
Training complete! Final model saved after epoch 30.


Computing threshold: 100%|████████████████████████████████████████████████████████| 1562/1562 [00:25<00:00, 60.42it/s]

Threshold set to 0.0000 based on 95th percentile of validation errors.
Checkpoint saved at checkpoints\unsupervised_final_model_with_threshold.pth
StandardScaler saved to scaler.pkl

Unsupervised model training run complete.
Model has been saved in the checkpoints directory.


## Model Evaluation

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyreadr
import torch
import gc
import pickle
import os
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Set device for PyTorch (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load datasets
df_test_faultfree = pyreadr.read_r("TEP_FaultFree_Testing.RData")['fault_free_testing']
df_test_faulty = pyreadr.read_r("TEP_Faulty_Testing.RData")['faulty_testing']

# Get all fault types
fault_types = sorted(df_test_faulty['faultNumber'].unique())
print(f"Available fault types: {fault_types}")

# Kalman Filter implementation for smoothing predictions (copied from original)
class KalmanFilter:
    """
    Simple implementation of Kalman filter for 1D signal smoothing
    """
    def __init__(self, dim_x, Q=1e-4, R=0.1):
        """
        Initialize Kalman filter

        Args:
            dim_x: State dimension
            Q: Process noise covariance
            R: Measurement noise covariance
        """
        self.dim_x = dim_x
        self.Q = Q
        self.R = R

    def filter(self, y_noisy):
        """
        Apply Kalman filtering to noisy observations

        Args:
            y_noisy: Noisy observations

        Returns:
            x_est: Filtered (smoothed) state estimates
        """
        n = len(y_noisy)
        x_est = np.zeros(n)  # State estimates
        P = np.zeros(n)      # Error covariance

        # Initial state and covariance
        if n > 0:
            x_est[0] = y_noisy[0]
            P[0] = 1.0
        else:
            return x_est # Return empty if no data

        for k in range(1, n):
            # Prediction step
            x_pred = x_est[k-1]      # State prediction
            P_pred = P[k-1] + self.Q  # Covariance prediction

            # Update step with measurement
            K = P_pred / (P_pred + self.R)  # Kalman gain
            x_est[k] = x_pred + K * (y_noisy[k] - x_pred)  # State update
            P[k] = (1 - K) * P_pred    # Covariance update

        return x_est

# Create PyTorch dataset - exactly as in original code
class UnsupervisedDataset(torch.utils.data.Dataset):
    """
    Dataset for unsupervised learning (no labels)
    """
    def __init__(self, X):
        self.X = X

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]

# Create sliding window time series input - using original implementation
def create_time_series_windows(X, lookback=15, max_samples=None, start_idx=0):
    """
    Create sliding window dataset for unsupervised learning
    with memory efficiency optimizations and proper handling of time order
    """
    # Calculate valid range for window creation (respecting time boundaries)
    n_potential_samples = len(X) - lookback + 1 - start_idx
    # Ensure n_potential_samples is not negative
    n_samples = max(0, n_potential_samples)

    # Handle max_samples limit if specified
    if max_samples is not None and n_samples > max_samples:
        n_samples = max_samples
    
    # Pre-allocate memory for efficiency
    n_features = X.shape[1]
    X_windows = np.zeros((n_samples, lookback, n_features), dtype=np.float32)
    
    # Process in batches to save memory
    for i in tqdm(range(n_samples), desc="Creating windows", miniters=int(n_samples/100) if n_samples > 100 else 1, leave=False):
        actual_idx = start_idx + i
        X_windows[i] = X[actual_idx : actual_idx + lookback]
    
    return torch.tensor(X_windows, dtype=torch.float32)

# Positional Encoding for Transformer model - exactly as in original code
class PositionalEncoding(torch.nn.Module):
    """
    Positional encoding for Transformer model
    Adds information about position of tokens in the sequence
    """
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)
        
        # Create positional encoding
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        """
        Add positional encoding to input
        
        Args:
            x: Input tensor of shape (batch_size, seq_length, d_model)
            
        Returns:
            Tensor with positional encoding added
        """
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

# Transformer-based autoencoder for unsupervised anomaly detection - exactly as in original code
class TransformerAutoencoder(torch.nn.Module):
    """
    Transformer-based autoencoder for unsupervised anomaly detection
    """
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2, dropout=0.1):
        """
        Initialize Transformer autoencoder
        
        Args:
            input_dim: Input feature dimension
            d_model: Hidden dimension for the model
            nhead: Number of attention heads
            num_layers: Number of transformer encoder layers
            dropout: Dropout probability
        """
        super().__init__()
        
        # Input embedding
        self.embedding = torch.nn.Linear(input_dim, d_model)
        
        # Positional encoding
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        # Transformer encoder layers
        encoder_layer = torch.nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=256, 
            dropout=dropout, 
            batch_first=True
        )
        
        self.transformer_encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Decoder to reconstruct the input
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(d_model, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, input_dim)
        )
        
    def forward(self, x):
        """
        Forward pass of the model
        
        Args:
            x: Input tensor of shape (batch_size, seq_length, input_dim)
            
        Returns:
            x_reconstructed: Reconstructed input
        """
        batch_size, seq_len, _ = x.size()
        
        # Embedding and scale
        x_embedded = self.embedding(x) * np.sqrt(self.embedding.out_features)
        
        # Add positional encoding
        x_embedded = self.pos_encoder(x_embedded)
        
        # Pass through transformer encoder
        x_encoded = self.transformer_encoder(x_embedded)
        
        # Decode each time step
        x_reconstructed = torch.zeros_like(x)
        for i in range(seq_len):
            x_reconstructed[:, i] = self.decoder(x_encoded[:, i])
            
        return x_reconstructed
    
    def get_reconstruction_error(self, x):
        """
        Calculate reconstruction error (MSE) for anomaly detection
        with memory-efficient implementation
        
        Args:
            x: Input tensor
            
        Returns:
            error: Reconstruction error
        """
        # Process one sample at a time if batch size is 1
        if x.size(0) == 1:
            x_reconstructed = self(x)
            error = torch.mean(torch.pow(x - x_reconstructed, 2), dim=(1, 2))
            return error
        
        # For larger batches, split into smaller sub-batches if needed
        batch_size = x.size(0)
        if batch_size > 4:  # If batch is large
            errors = []
            sub_batch_size = 4  # Process 4 samples at a time
            
            for i in range(0, batch_size, sub_batch_size):
                end_idx = min(i + sub_batch_size, batch_size)
                sub_batch = x[i:end_idx]
                sub_reconstructed = self(sub_batch)
                sub_error = torch.mean(torch.pow(sub_batch - sub_reconstructed, 2), dim=(1, 2))
                errors.append(sub_error)
                
                # Free memory
                del sub_batch, sub_reconstructed
                
            return torch.cat(errors)
        else:
            # Standard processing for reasonable batch sizes
            x_reconstructed = self(x)
            error = torch.mean(torch.pow(x - x_reconstructed, 2), dim=(1, 2))
            return error

# Neural network anomaly detector with reconstruction-based approach - exactly as in original code
class UnsupervisedAnomalyDetector:
    """
    Unsupervised anomaly detector based on reconstruction error
    """
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2, dropout=0.1, Q=1e-4, R=0.1):
        """
        Initialize detector
        
        Args:
            input_dim: Input feature dimension
            d_model: Hidden dimension for transformer
            nhead: Number of attention heads
            num_layers: Number of transformer layers
            dropout: Dropout probability
            Q: Process noise for Kalman filter
            R: Measurement noise for Kalman filter
        """
        self.autoencoder = TransformerAutoencoder(input_dim, d_model, nhead, num_layers, dropout)
        self.kalman_filter = KalmanFilter(1, Q, R)
        self.device = device
        self.autoencoder.to(self.device)
        
        # Training related variables
        self.train_losses = []
        self.val_losses = []
        self.epochs = []
        self.current_epoch = 0 # Stores the last completed epoch (0-indexed)
        
        # Threshold for anomaly detection
        self.threshold = None
        
        # Checkpoint directory
        self.checkpoint_dir = 'checkpoints'
        os.makedirs(self.checkpoint_dir, exist_ok=True)
        
    def save_checkpoint(self, epoch, optimizer, loss, filename=None):
        """
        Save model checkpoint - safely handles None optimizer
        
        Args:
            epoch: Current epoch (1-based for saving convention)
            optimizer: Optimizer state (can be None for inference-only checkpoints)
            loss: Current loss value
            filename: Name of the checkpoint file
        """
        if filename is None:
            filename = f'unsupervised_checkpoint.pth'
        path = os.path.join(self.checkpoint_dir, filename)
    
        # Create checkpoint dictionary
        checkpoint_dict = {
            'epoch': epoch, # This is the 1-based epoch number
            'model_state_dict': self.autoencoder.state_dict(),
            'loss': loss,
            'train_losses': self.train_losses,
            'val_losses': self.val_losses,
            'epochs': self.epochs, # List of 1-based epoch numbers trained
            'current_epoch': self.current_epoch, # Last completed 0-indexed epoch
            'threshold': self.threshold
        }
    
        # Only add optimizer state if it's not None
        if optimizer is not None:
            try:
                checkpoint_dict['optimizer_state_dict'] = optimizer.state_dict()
            except AttributeError:
                print("Warning: optimizer does not have state_dict method, not saving optimizer state")
    
        torch.save(checkpoint_dict, path)
        print(f"Checkpoint saved at {path}")
        
    def load_checkpoint(self, optimizer, filename=None):
        """
        Load model checkpoint
        
        Args:
            optimizer: Optimizer to load state into (can be None)
            filename: Name of the checkpoint file
            
        Returns:
            start_epoch: Epoch to start training from (0-indexed for loop)
            loss: Loss value of the checkpoint
        """
        if filename is None:
            filename = f'unsupervised_checkpoint.pth'
        path = os.path.join(self.checkpoint_dir, filename)
        if os.path.exists(path):
            checkpoint = torch.load(path, map_location=self.device)
            self.autoencoder.load_state_dict(checkpoint['model_state_dict'])
            
            if optimizer is not None and 'optimizer_state_dict' in checkpoint:
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                
            self.train_losses = checkpoint.get('train_losses', [])
            self.val_losses = checkpoint.get('val_losses', [])
            self.epochs = checkpoint.get('epochs', [])
            self.current_epoch = checkpoint.get('current_epoch', 0) # Last completed 0-indexed epoch
            self.threshold = checkpoint.get('threshold', None)
            
            # 'epoch' in checkpoint is 1-based, so start_epoch for training loop is this value
            # current_epoch (0-indexed) is more reliable for determining next epoch.
            start_epoch_for_loop = self.current_epoch + 1 if self.epochs else 0

            print(f"Checkpoint loaded from {path}, resuming from epoch {start_epoch_for_loop}")
            return start_epoch_for_loop, checkpoint.get('loss', float('inf'))
        else:
            print(f"No checkpoint found at {path}, starting from scratch")
            return 0, float('inf') # Start from epoch 0 (0-indexed)
        
    def predict(self, X, batch_size=4):
        """
        Make predictions using the reconstruction error and apply Kalman filtering
        with memory-efficient batch processing
        
        Args:
            X: Input data tensor or numpy array
            batch_size: Batch size for prediction to avoid OOM errors
            
        Returns:
            anomalies: Binary anomaly predictions
            filtered_scores: Continuous anomaly scores after Kalman filtering
            raw_scores: Raw reconstruction errors
        """
        # Using same logic as original code
        if self.threshold is None:
            self.autoencoder.eval()
            print("Warning: UnsupervisedAnomalyDetector.predict called with self.threshold = None. Setting default threshold of 0.1")
            self.threshold = 0.1  # Set a default threshold instead of raising an error
        
        self.autoencoder.eval()

        if isinstance(X, np.ndarray):
            total_samples = X.shape[0]
        elif isinstance(X, torch.Tensor):
            total_samples = X.size(0)
        else:
            raise TypeError("Input X must be a NumPy array or PyTorch Tensor.")

        if total_samples == 0:
            return np.array([]), np.array([]), np.array([])

        all_errors = np.zeros(total_samples)
        
        try:
            with torch.no_grad():
                for i in tqdm(range(0, total_samples, batch_size), desc="Predicting", leave=False):
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    
                    end_idx = min(i + batch_size, total_samples)
                    if isinstance(X, np.ndarray):
                        X_batch_np = X[i:end_idx]
                        X_tensor = torch.tensor(X_batch_np, dtype=torch.float32).to(self.device)
                    else: # X is already a Tensor
                        X_tensor = X[i:end_idx].to(self.device)
                    
                    reconstruction_errors = self.autoencoder.get_reconstruction_error(X_tensor)
                    all_errors[i:end_idx] = reconstruction_errors.cpu().numpy()
                    
                    del X_tensor, reconstruction_errors
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
            
            filtered_scores = self.kalman_filter.filter(all_errors)
            anomalies = filtered_scores > self.threshold
            
            return anomalies, filtered_scores, all_errors
            
        except Exception as e:
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            raise e

# Enhanced evaluate_fault_detector function - modified to match original implementation
def evaluate_fault_detector(detector, fault_type, df_test_faultfree, df_test_faulty, scaler, lookback=15, batch_size=4, 
                           save_plots=True, output_dir='results'):
    """
    Enhanced evaluate anomaly detector with detailed metrics and visualizations
    """
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\n{'='*80}")
    print(f"Evaluating detector on Fault Type {fault_type}")
    print(f"{'='*80}")
    
    try:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        X_test_data_raw = None
        y_true_raw = None

        if fault_type == 0:
            # Use only fault-free test data
            X_test_data_raw = df_test_faultfree.iloc[:, 3:].values
            y_true_raw = np.zeros(len(X_test_data_raw))
            if len(X_test_data_raw) == 0:
                print(f"No fault-free test data available. Skipping evaluation for fault type 0.")
                return {'fault_type': fault_type, 'error': 'No fault-free test data', 
                        'precision': 0, 'recall': 0, 'f1': 0, 'accuracy': 0, 'roc_auc': np.nan, 'pr_auc': np.nan}
        else:
            # Use specific fault data from faulty test set
            test_fault_data_df = df_test_faulty[df_test_faulty['faultNumber'] == fault_type]
            if len(test_fault_data_df) == 0:
                print(f"No test data found for fault type {fault_type}. Skipping evaluation...")
                return {'fault_type': fault_type, 'error': f'No test data for fault {fault_type}', 
                        'precision': 0, 'recall': 0, 'f1': 0, 'accuracy': 0, 'roc_auc': np.nan, 'pr_auc': np.nan}

            X_test_data_raw = test_fault_data_df.iloc[:, 3:].values
            y_true_raw = np.ones(len(X_test_data_raw))
            # First 160 samples in TEP faulty datasets are normal before fault inception
            if len(y_true_raw) >= 160: # Ensure there are enough samples
                 y_true_raw[:160] = 0
            else: # If dataset is shorter than 160, all are normal if fault hasn't started.
                  pass

        X_test_scaled = scaler.transform(X_test_data_raw)
        # The number of windows will be len(X_test_scaled) - lookback + 1 for start_idx=0
        X_test_windows = create_time_series_windows(X_test_scaled, lookback, start_idx=0)

        if X_test_windows.nelement() == 0: # Check if tensor is empty
            print(f"Not enough data to create windows for fault type {fault_type} with lookback {lookback}. Skipping.")
            return {'fault_type': fault_type, 'error': 'Not enough data for windows', 
                    'precision': 0, 'recall': 0, 'f1': 0, 'accuracy': 0, 'roc_auc': np.nan, 'pr_auc': np.nan}

        # Adjust labels: label for window X[i:i+lookback-1] corresponds to label y[i+lookback-1]
        # This means we need labels starting from index (lookback-1) to match the windows
        y_true_for_windows = y_true_raw[lookback-1:lookback-1+len(X_test_windows)]
        
        num_windows = len(X_test_windows)
        
        if len(y_true_for_windows) < num_windows:
            print(f"Warning: Original labels too short ({len(y_true_for_windows)}) for {num_windows} windows for fault {fault_type}. Trimming windows.")
            X_test_windows = X_test_windows[:len(y_true_for_windows)]
            num_windows = len(X_test_windows)
            if num_windows == 0:
                 print(f"No windows left after label alignment for fault {fault_type}.")
                 return {'fault_type': fault_type, 'error': 'No windows after label alignment', 
                        'precision': 0, 'recall': 0, 'f1': 0, 'accuracy': 0, 'roc_auc': np.nan, 'pr_auc': np.nan}

        # Get predictions and scores
        anomalies_pred, filtered_scores, raw_scores = detector.predict(
            X_test_windows, 
            batch_size=batch_size
        )
        
        # Calculate confusion matrix components
        tn, fp, fn, tp = confusion_matrix(y_true_for_windows, anomalies_pred, labels=[0, 1]).ravel()
        
        # Calculate evaluation metrics
        precision, recall, f1, _ = precision_recall_fscore_support(y_true_for_windows, anomalies_pred, average='binary', zero_division=0)
        accuracy = accuracy_score(y_true_for_windows, anomalies_pred)
        
        roc_auc = np.nan
        pr_auc = np.nan

        if len(np.unique(y_true_for_windows)) > 1: # Check if more than one class in true labels
            if len(y_true_for_windows) == len(filtered_scores):
                try:
                    roc_auc = roc_auc_score(y_true_for_windows, filtered_scores)
                    precision_curve_vals, recall_curve_vals, _ = precision_recall_curve(y_true_for_windows, filtered_scores)
                    pr_auc = auc(recall_curve_vals, precision_curve_vals)
                except ValueError as e_auc: # Handles cases like all scores being identical for one class
                    print(f"Could not calculate ROC/PR AUC for fault {fault_type}: {e_auc}")
            else:
                print(f"Warning: Length mismatch for ROC/PR AUC calculation for fault {fault_type}. True: {len(y_true_for_windows)}, Scores: {len(filtered_scores)}")
        else:
            print(f"Warning: Only one class present in y_true for fault type {fault_type}. ROC AUC and PR AUC are not defined.")

        # Calculate detailed metrics
        total_predictions = len(y_true_for_windows)
        total_actual_positives = np.sum(y_true_for_windows)
        total_actual_negatives = total_predictions - total_actual_positives
        
        results = {
            'fault_type': fault_type,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'accuracy': accuracy,
            'roc_auc': roc_auc,
            'pr_auc': pr_auc,
            'true_positives': tp,
            'false_positives': fp,
            'true_negatives': tn,
            'false_negatives': fn,
            'total_actual_positives': total_actual_positives,
            'total_actual_negatives': total_actual_negatives,
            'total_predictions': total_predictions
        }

        # Print detailed results including confusion matrix details
        print(f"\nResults for Fault Type {fault_type}:")
        print(f"  Precision: {results['precision']:.4f}")
        print(f"  Recall:    {results['recall']:.4f}")
        print(f"  F1 Score:  {results['f1']:.4f}")
        print(f"  Accuracy:  {results['accuracy']:.4f}")
        print(f"  ROC-AUC:   {results['roc_auc']:.4f}")
        print(f"  PR-AUC:    {results['pr_auc']:.4f}")
        
        print(f"\nConfusion Matrix Details for Fault Type {fault_type}:")
        print(f"  Total samples: {total_predictions}")
        print(f"  True Positives (TP): {tp} ({tp/total_predictions*100:.1f}%)")
        print(f"  False Positives (FP): {fp} ({fp/total_predictions*100:.1f}%)")
        print(f"  True Negatives (TN): {tn} ({tn/total_predictions*100:.1f}%)")
        print(f"  False Negatives (FN): {fn} ({fn/total_predictions*100:.1f}%)")
        
        if total_actual_positives > 0:
            print(f"  TP Rate (Recall): {tp/total_actual_positives:.4f}")
            print(f"  FN Rate: {fn/total_actual_positives:.4f}")
        if total_actual_negatives > 0:
            print(f"  TN Rate (Specificity): {tn/total_actual_negatives:.4f}")
            print(f"  FP Rate: {fp/total_actual_negatives:.4f}")

        if save_plots and len(filtered_scores) > 0:
            # Save visualizations
            
            # Visualization of filtered scores and predictions
            # Similar to original but with enhanced formatting
            max_points = 1000
            if len(filtered_scores) > max_points:
                idx = np.linspace(0, len(filtered_scores)-1, max_points, dtype=int)
                vis_filtered_scores = filtered_scores[idx]
                vis_raw_scores = raw_scores[idx]
                vis_anomalies = anomalies_pred[idx]
                vis_y_test = y_true_for_windows[idx]
            else:
                vis_filtered_scores = filtered_scores
                vis_raw_scores = raw_scores
                vis_anomalies = anomalies_pred
                vis_y_test = y_true_for_windows
            
            plt.figure(figsize=(16, 10))
            
            # Top subplot: Reconstruction errors and filtered scores
            plt.subplot(3, 1, 1)
            plt.plot(vis_raw_scores, 'y-', alpha=0.3, label='Raw Reconstruction Error')
            plt.plot(vis_filtered_scores, 'b-', linewidth=1.5, label='Filtered Anomaly Score')
            
            # Mark detected anomalies on the filtered score plot
            anomaly_indices = np.where(vis_anomalies)[0]
            if len(anomaly_indices) > 0:
                plt.scatter(anomaly_indices, vis_filtered_scores[anomaly_indices], 
                           color='r', s=20, marker='x', label='Detected Anomalies')
            
            # Add threshold line
            if detector.threshold is not None:
                plt.axhline(detector.threshold, color='k', linestyle='--', 
                           label=f'Threshold = {detector.threshold:.4f}')
            
            plt.title(f'Fault Type {fault_type} - Reconstruction Errors and Anomaly Scores', fontsize=14)
            plt.ylabel('Error Magnitude')
            plt.grid(True, alpha=0.3)
            plt.legend(loc='upper right')
            
            # Middle subplot: True vs Predicted Labels
            plt.subplot(3, 1, 2)
            plt.plot(vis_y_test, 'g-', linewidth=2, label='True Labels')
            plt.plot(vis_anomalies, 'r--', linewidth=1.5, label='Predicted Labels')
            
            # Highlight false positives (blue) and false negatives (orange)
            fp_indices = np.where((vis_y_test == 0) & (vis_anomalies == 1))[0]
            fn_indices = np.where((vis_y_test == 1) & (vis_anomalies == 0))[0]
            
            if len(fp_indices) > 0:
                plt.scatter(fp_indices, vis_anomalies[fp_indices], color='blue', s=30, 
                           marker='o', label='False Positives', alpha=0.5)
            
            if len(fn_indices) > 0:
                plt.scatter(fn_indices, vis_y_test[fn_indices], color='orange', s=30, 
                           marker='o', label='False Negatives', alpha=0.5)
                
            plt.title(f'True vs Predicted Labels', fontsize=14)
            plt.ylabel('Label (0=Normal, 1=Fault)')
            plt.ylim(-0.1, 1.1)
            plt.yticks([0, 1])
            plt.grid(True, alpha=0.3)
            plt.legend(loc='upper right')
            
            # Bottom subplot: Error distribution by label
            plt.subplot(3, 1, 3)
            
            # Create bins for the histogram
            bins = np.linspace(0, max(vis_filtered_scores) * 1.1, 50)
            
            # Separate scores by true label
            normal_indices = np.where(vis_y_test == 0)[0]
            fault_indices = np.where(vis_y_test == 1)[0]
            
            if len(normal_indices) > 0:
                normal_scores = vis_filtered_scores[normal_indices]
                plt.hist(normal_scores, bins=bins, alpha=0.6, color='green', 
                        label=f'Normal ({len(normal_indices)} samples)')
            
            if len(fault_indices) > 0:
                fault_scores = vis_filtered_scores[fault_indices]
                plt.hist(fault_scores, bins=bins, alpha=0.6, color='red', 
                        label=f'Fault ({len(fault_indices)} samples)')
            
            # Add threshold line
            if detector.threshold is not None:
                plt.axvline(detector.threshold, color='k', linestyle='--', 
                           label=f'Threshold = {detector.threshold:.4f}')
            
            plt.title('Distribution of Filtered Scores by True Label', fontsize=14)
            plt.xlabel('Filtered Anomaly Score')
            plt.ylabel('Frequency')
            plt.grid(True, alpha=0.3)
            plt.legend(loc='upper right')
            
            plt.tight_layout()
            plt.savefig(f'{output_dir}/fault_{fault_type}_detailed_eval.png', dpi=150, bbox_inches='tight')
            plt.close()
            
            print(f"Visualizations for Fault Type {fault_type} saved to {output_dir}/")
        
        # Clean up memory
        del X_test_data_raw, X_test_scaled, X_test_windows
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
        return results
    
    except Exception as e:
        print(f"Error evaluating fault type {fault_type}: {str(e)}")
        import traceback
        traceback.print_exc()
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        return {'fault_type': fault_type, 'error': str(e), 
                'precision': 0, 'recall': 0, 'f1': 0, 'accuracy': 0, 'roc_auc': np.nan, 'pr_auc': np.nan}

def load_model_and_scaler():
    """
    Load the trained model and scaler from files
    """
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load scaler
    try:
        with open('scaler.pkl', 'rb') as f:
            scaler = pickle.load(f)
        print("Loaded StandardScaler from scaler.pkl")
    except FileNotFoundError:
        print("Error: scaler.pkl file not found. Please run training first.")
        return None, None
    
    # Determine feature dimension from test data
    feature_dim = df_test_faultfree.iloc[:, 3:].shape[1]
    
    # Create detector instance with same parameters as original
    detector = UnsupervisedAnomalyDetector(
        input_dim=feature_dim, d_model=64, nhead=4, num_layers=2, dropout=0.1, Q=1e-5, R=0.1
    )
    
    # Load model checkpoints
    try:
        # First try to load the checkpoint with threshold
        checkpoint_path = os.path.join('checkpoints', 'unsupervised_final_model_with_threshold.pth')
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path, map_location=device)
            detector.autoencoder.load_state_dict(checkpoint['model_state_dict'])
            detector.threshold = checkpoint.get('threshold', None)
            print(f"Loaded model from {checkpoint_path}")
            print(f"Model threshold: {detector.threshold}")
            return detector, scaler
            
        # Next try best model
        checkpoint_path = os.path.join('checkpoints', 'unsupervised_best_model.pth')
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path, map_location=device)
            detector.autoencoder.load_state_dict(checkpoint['model_state_dict'])
            detector.threshold = checkpoint.get('threshold', None)
            print(f"Loaded model from {checkpoint_path}")
            print(f"Model threshold: {detector.threshold}")
            return detector, scaler
        
        # Then try final model
        checkpoint_path = os.path.join('checkpoints', 'unsupervised_final_model.pth')
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path, map_location=device)
            detector.autoencoder.load_state_dict(checkpoint['model_state_dict'])
            detector.threshold = checkpoint.get('threshold', None)
            print(f"Loaded model from {checkpoint_path}")
            print(f"Model threshold: {detector.threshold}")
            return detector, scaler
        
        # If none of the above found, try any checkpoint
        for file in os.listdir('checkpoints'):
            if file.endswith('.pth'):
                checkpoint_path = os.path.join('checkpoints', file)
                checkpoint = torch.load(checkpoint_path, map_location=device)
                detector.autoencoder.load_state_dict(checkpoint['model_state_dict'])
                detector.threshold = checkpoint.get('threshold', None)
                print(f"Loaded model from {checkpoint_path}")
                print(f"Model threshold: {detector.threshold}")
                return detector, scaler
        
        print("Error: No model checkpoint found in the checkpoints directory.")
        return None, None
        
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None

def set_threshold_from_normal_data(detector, scaler, lookback=15, percentile=95, batch_size=2):
    """
    Set threshold from normal operation data - implements same logic as in the original code
    """
    print(f"Computing threshold based on normal operation data (percentile={percentile})...")
    
    # Use a portion of fault-free test data to calculate threshold
    max_samples = 5000  # Limit samples to avoid memory issues
    X_test_normal = df_test_faultfree.iloc[:, 3:].values
    if len(X_test_normal) > max_samples:
        X_test_normal = X_test_normal[:max_samples]
    
    X_test_normal_scaled = scaler.transform(X_test_normal)
    X_test_normal_windows = create_time_series_windows(X_test_normal_scaled, lookback)
    
    errors = []
    detector.autoencoder.eval()
    
    with torch.no_grad():
        for i in tqdm(range(0, len(X_test_normal_windows), batch_size), desc="Computing threshold"):
            end_idx = min(i + batch_size, len(X_test_normal_windows))
            X_batch = X_test_normal_windows[i:end_idx].to(device)
            
            reconstruction_errors = detector.autoencoder.get_reconstruction_error(X_batch)
            errors.extend(reconstruction_errors.cpu().numpy())
            
            del X_batch, reconstruction_errors
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
    
    # Set threshold at 95th percentile (or specified percentile)
    if errors:
        threshold = np.percentile(errors, percentile)
        print(f"Threshold set to {threshold:.4f} based on {percentile}th percentile of normal operation data")
        detector.threshold = threshold
    else:
        print("Warning: Could not compute errors for threshold. Using default.")
        detector.threshold = 0.1
    
    return detector.threshold

def evaluate_all_faults(detector, df_test_faultfree, df_test_faulty, scaler, lookback=15, batch_size=2,
                         skip_faults=None, output_dir='results'):
    """
    Evaluate detector on all fault types and generate summary
    """
    if skip_faults is None:
        skip_faults = [3, 9, 15]  # Default skipped faults as in original code
        
    # Define fault types for evaluation: 0 and 1-20 excluding skipped faults
    all_dataset_fault_numbers = sorted(df_test_faulty['faultNumber'].unique())
    fault_types_to_evaluate = [0]  # Start with fault type 0 (normal)
    
    for ft in all_dataset_fault_numbers:
        if ft not in skip_faults:
            fault_types_to_evaluate.append(ft)
    
    print(f"Will evaluate for fault types: {fault_types_to_evaluate}")
    os.makedirs(output_dir, exist_ok=True)
    
    # Store results for all fault types
    all_results = {}
    
    # Evaluate each fault type
    for fault_type in fault_types_to_evaluate:
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
        result = evaluate_fault_detector(
            detector=detector,
            fault_type=fault_type,
            df_test_faultfree=df_test_faultfree, 
            df_test_faulty=df_test_faulty,    
            scaler=scaler,
            lookback=lookback,
            batch_size=batch_size,
            output_dir=output_dir
        )
        
        if result:
            all_results[fault_type] = result
    
    # Create summary table
    summary_data = []
    for fault_type, result in all_results.items():
        summary_data.append({
            'Fault Type': fault_type,
            'Precision': result.get('precision', 0),
            'Recall': result.get('recall', 0),
            'F1 Score': result.get('f1', 0),
            'Accuracy': result.get('accuracy', 0),
            'ROC-AUC': result.get('roc_auc', np.nan),
            'PR-AUC': result.get('pr_auc', np.nan),
            'TP': result.get('true_positives', 0),
            'FP': result.get('false_positives', 0),
            'TN': result.get('true_negatives', 0),
            'FN': result.get('false_negatives', 0)
        })
    
    # Save summary results to CSV and display
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(f"{output_dir}/fault_detection_summary.csv", index=False)
    print("\nSummary of results for all evaluated fault types:")
    print(summary_df)
    
    # Create summary visualizations
    if len(summary_data) > 0:
        # Bar chart of F1 scores by fault type
        plt.figure(figsize=(14, 8))
        fault_types = summary_df['Fault Type'].values
        f1_scores = summary_df['F1 Score'].values
        
        # Sort by fault type
        sort_idx = np.argsort(fault_types)
        fault_types = fault_types[sort_idx]
        f1_scores = f1_scores[sort_idx]
        
        bars = plt.bar(fault_types, f1_scores, color='skyblue')
        
        # Add value labels on top of bars
        for i, bar in enumerate(bars):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.4f}', ha='center', va='bottom', rotation=0, fontsize=9)
        
        plt.title('F1 Scores by Fault Type', fontsize=16)
        plt.xlabel('Fault Type', fontsize=14)
        plt.ylabel('F1 Score', fontsize=14)
        plt.xticks(fault_types)
        plt.ylim(0, 1.1)
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig(f"{output_dir}/summary_f1_scores.png", dpi=150, bbox_inches='tight')
        plt.close()
        
        # Create confusion matrix components visualization
        plt.figure(figsize=(14, 10))
        
        # Create subplots for TP, FP, TN, FN
        plt.subplot(2, 2, 1)
        plt.bar(fault_types, summary_df['TP'].values[sort_idx], color='green')
        plt.title('True Positives by Fault Type')
        plt.xlabel('Fault Type')
        plt.ylabel('Count')
        plt.xticks(fault_types)
        plt.grid(axis='y', alpha=0.3)
        
        plt.subplot(2, 2, 2)
        plt.bar(fault_types, summary_df['FP'].values[sort_idx], color='red')
        plt.title('False Positives by Fault Type')
        plt.xlabel('Fault Type')
        plt.ylabel('Count')
        plt.xticks(fault_types)
        plt.grid(axis='y', alpha=0.3)
        
        plt.subplot(2, 2, 3)
        plt.bar(fault_types, summary_df['TN'].values[sort_idx], color='blue')
        plt.title('True Negatives by Fault Type')
        plt.xlabel('Fault Type')
        plt.ylabel('Count')
        plt.xticks(fault_types)
        plt.grid(axis='y', alpha=0.3)
        
        plt.subplot(2, 2, 4)
        plt.bar(fault_types, summary_df['FN'].values[sort_idx], color='orange')
        plt.title('False Negatives by Fault Type')
        plt.xlabel('Fault Type')
        plt.ylabel('Count')
        plt.xticks(fault_types)
        plt.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/summary_confusion_components.png", dpi=150, bbox_inches='tight')
        plt.close()
    
    print(f"\nAll summary visualizations saved to {output_dir}/")
    return all_results

def main_testing():
    """
    Main function for testing - modified to match original implementation
    """
    lookback = 15
    batch_size = 2  # Smaller batch size for testing to avoid memory issues
    output_dir = 'fault_evaluation_results'
    
    # Load model and scaler
    detector, scaler = load_model_and_scaler()
    
    if detector is None or scaler is None:
        print("Error: Could not load model or scaler. Please run training first.")
        return
    
    # Ensure threshold is set correctly
    if detector.threshold is None or detector.threshold < 0.001:
        print("Threshold not set or invalid. Setting threshold from normal data.")
        set_threshold_from_normal_data(detector, scaler, lookback=lookback, percentile=95, batch_size=batch_size)
    
    print(f"Using threshold: {detector.threshold}")
    
    # Evaluate all fault types
    results = evaluate_all_faults(
        detector=detector,
        df_test_faultfree=df_test_faultfree,
        df_test_faulty=df_test_faulty,
        scaler=scaler,
        lookback=lookback,
        batch_size=batch_size,
        output_dir=output_dir
    )
    
    print("\nTesting completed successfully!")
    return results

if __name__ == "__main__":
    results = main_testing()

Using device: cuda
Available fault types: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Loaded StandardScaler from scaler.pkl
Loaded model from checkpoints\unsupervised_final_model_with_threshold.pth
Model threshold: 4.132619324082043e-05
Threshold not set or invalid. Setting threshold from normal data.
Computing threshold based on normal operation data (percentile=95)...


Computing threshold: 100%|███████████████████████████████████████████████████████| 2493/2493 [00:07<00:00, 317.90it/s]


Threshold set to 0.0000 based on 95th percentile of normal operation data
Using threshold: 3.307684892206453e-05
Will evaluate for fault types: [0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20]

Evaluating detector on Fault Type 0



Results for Fault Type 0:
  Precision: 0.0000
  Recall:    0.0000
  F1 Score:  0.0000
  Accuracy:  0.9752
  ROC-AUC:   nan
  PR-AUC:    nan

Confusion Matrix Details for Fault Type 0:
  Total samples: 479986
  True Positives (TP): 0 (0.0%)
  False Positives (FP): 11906 (2.5%)
  True Negatives (TN): 468080 (97.5%)
  False Negatives (FN): 0 (0.0%)
  TN Rate (Specificity): 0.9752
  FP Rate: 0.0248
Visualizations for Fault Type 0 saved to fault_evaluation_results/

Evaluating detector on Fault Type 1



Results for Fault Type 1:
  Precision: 1.0000
  Recall:    1.0000
  F1 Score:  1.0000
  Accuracy:  1.0000
  ROC-AUC:   1.0000
  PR-AUC:    1.0000

Confusion Matrix Details for Fault Type 1:
  Total samples: 479986
  True Positives (TP): 479830 (100.0%)
  False Positives (FP): 0 (0.0%)
  True Negatives (TN): 146 (0.0%)
  False Negatives (FN): 10 (0.0%)
  TP Rate (Recall): 1.0000
  FN Rate: 0.0000
  TN Rate (Specificity): 1.0000
  FP Rate: 0.0000
Visualizations for Fault Type 1 saved to fault_evaluation_results/

Evaluating detector on Fault Type 2


...


Results for Fault Type 16:
  Precision: 1.0000
  Recall:    1.0000
  F1 Score:  1.0000
  Accuracy:  1.0000
  ROC-AUC:   1.0000
  PR-AUC:    1.0000

Confusion Matrix Details for Fault Type 16:
  Total samples: 479986
  True Positives (TP): 479825 (100.0%)
  False Positives (FP): 0 (0.0%)
  True Negatives (TN): 146 (0.0%)
  False Negatives (FN): 15 (0.0%)
  TP Rate (Recall): 1.0000
  FN Rate: 0.0000
  TN Rate (Specificity): 1.0000
  FP Rate: 0.0000
Visualizations for Fault Type 16 saved to fault_evaluation_results/

Evaluating detector on Fault Type 17



Results for Fault Type 17:
  Precision: 1.0000
  Recall:    0.9998
  F1 Score:  0.9999
  Accuracy:  0.9998
  ROC-AUC:   0.9999
  PR-AUC:    1.0000

Confusion Matrix Details for Fault Type 17:
  Total samples: 479986
  True Positives (TP): 479757 (100.0%)
  False Positives (FP): 0 (0.0%)
  True Negatives (TN): 146 (0.0%)
  False Negatives (FN): 83 (0.0%)
  TP Rate (Recall): 0.9998
  FN Rate: 0.0002
  TN Rate (Specificity): 1.0000
  FP Rate: 0.0000
Visualizations for Fault Type 17 saved to fault_evaluation_results/

Evaluating detector on Fault Type 18



Results for Fault Type 18:
  Precision: 1.0000
  Recall:    0.9999
  F1 Score:  0.9999
  Accuracy:  0.9999
  ROC-AUC:   0.9999
  PR-AUC:    1.0000

Confusion Matrix Details for Fault Type 18:
  Total samples: 479986
  True Positives (TP): 479769 (100.0%)
  False Positives (FP): 0 (0.0%)
  True Negatives (TN): 146 (0.0%)
  False Negatives (FN): 71 (0.0%)
  TP Rate (Recall): 0.9999
  FN Rate: 0.0001
  TN Rate (Specificity): 1.0000
  FP Rate: 0.0000
Visualizations for Fault Type 18 saved to fault_evaluation_results/

Evaluating detector on Fault Type 19



Results for Fault Type 19:
  Precision: 1.0000
  Recall:    1.0000
  F1 Score:  1.0000
  Accuracy:  1.0000
  ROC-AUC:   1.0000
  PR-AUC:    1.0000

Confusion Matrix Details for Fault Type 19:
  Total samples: 479986
  True Positives (TP): 479821 (100.0%)
  False Positives (FP): 0 (0.0%)
  True Negatives (TN): 146 (0.0%)
  False Negatives (FN): 19 (0.0%)
  TP Rate (Recall): 1.0000
  FN Rate: 0.0000
  TN Rate (Specificity): 1.0000
  FP Rate: 0.0000
Visualizations for Fault Type 19 saved to fault_evaluation_results/

Evaluating detector on Fault Type 20



Results for Fault Type 20:
  Precision: 1.0000
  Recall:    0.9999
  F1 Score:  0.9999
  Accuracy:  0.9999
  ROC-AUC:   1.0000
  PR-AUC:    1.0000

Confusion Matrix Details for Fault Type 20:
  Total samples: 479986
  True Positives (TP): 479786 (100.0%)
  False Positives (FP): 0 (0.0%)
  True Negatives (TN): 146 (0.0%)
  False Negatives (FN): 54 (0.0%)
  TP Rate (Recall): 0.9999
  FN Rate: 0.0001
  TN Rate (Specificity): 1.0000
  FP Rate: 0.0000
Visualizations for Fault Type 20 saved to fault_evaluation_results/

Summary of results for all evaluated fault types:
    Fault Type  Precision    Recall  F1 Score  Accuracy   ROC-AUC  PR-AUC  \
0            0        0.0  0.000000  0.000000  0.975195       NaN     NaN   
1            1        1.0  0.999979  0.999990  0.999979  0.999995     1.0   
2            2        1.0  0.999958  0.999979  0.999958  0.999990     1.0   
3            4        1.0  0.972499  0.986058  0.972508  0.999967     1.0   
4            5        1.0  0.999983  0.99999